In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
L1 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_light_z1.csv', delim_whitespace=True)
L2 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_light_z2.csv', delim_whitespace=True)
L3 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_light_z3.csv', delim_whitespace=True)
L4 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_light_z3a.csv', delim_whitespace=True)

In [2]:
M1 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_movement_z1.csv', delim_whitespace=True)
M2 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_movement_z2.csv', delim_whitespace=True)
M3 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_movement_z3.csv', delim_whitespace=True)
M4 = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/babd_movement_z3a.csv', delim_whitespace=True)

In [3]:
# this is the df of april
forecast = pd.read_csv('/Users/Alessandro/Desktop/IOT-Project/forecast.csv') 

In [4]:
M4['zone'] = M4['zone'].apply(lambda x: x + 1)
L4['zone'] = L4['zone'].apply(lambda x: x + 1)

In [5]:
movement = pd.concat([M1, M2, M3, M4], axis=0)

In [6]:
light = pd.concat([L1,L2,L3,L4], axis=0)

In [7]:
light = light.sort_values(by=['zone'], ascending=True)

In [8]:
light.reset_index(inplace=True)

In [9]:
pd.set_option("display.max_rows",20)

In [10]:
light["index"] = pd.to_datetime(light["index"], errors="coerce")

In [11]:
light["week"] = light["index"].dt.isocalendar().week

In [12]:
light["timestamp"] = pd.to_datetime(light["timestamp"],errors="coerce")

In [13]:
light["hour"] = light["timestamp"].dt.hour

In [14]:
newcol = []
for i in light['value']:
    if i<450:
        newcol.append(i-450)
    elif i>650:
        newcol.append(i-650)
    else:
        newcol.append(0)

newcol = pd.DataFrame(newcol)
light['diff']=newcol

In [15]:
# why did I create this copy? 
prevision = light.copy()
prevision["day"] = prevision["index"].dt.day
prevision["hour"] = prevision["timestamp"].dt.hour

In [16]:
light.drop(["index","sensor","id","unit_of_measure","timestamp","zone"], axis=1, inplace=True)

In [17]:
prevision.drop(["index","sensor","id","unit_of_measure","timestamp","zone"], axis=1, inplace=True)

In [18]:
#light[light["value"]==2].shape[0]/light.shape[0]
#light["value"].value_counts()

In [19]:
weekwise = light.groupby(["week","hour"],as_index=False).mean()

#### With the next for loop I am displaying how the measure of the light has changed over the weeks for each hour


In [20]:
dicts = {}
for i in weekwise.index:
    if weekwise.iloc[i,1] not in dicts.keys():
        dicts[weekwise.iloc[i,1]] = [weekwise.iloc[i,2]]
    else:
        dicts[weekwise.iloc[i,1]].append(weekwise.iloc[i,2])

In [21]:
dicts

{1: [7.865584415584416,
  7.795671167593328,
  8.043760129659644,
  7.087638979725311,
  2.0],
 2: [7.735975741823695,
  7.777667395191735,
  8.094675925925927,
  7.172391233235198,
  2.0],
 3: [7.779510504656703,
  7.782720953326713,
  8.050729673384295,
  7.143510951291272,
  2.0],
 4: [7.743556421918995,
  7.795553791186979,
  7.995367153115589,
  7.116704805491991,
  2.0],
 5: [7.813001083423619,
  7.832605242255759,
  8.09708063021316,
  7.44426283099052,
  3.033333333333333],
 6: [19.59883091578264,
  38.59078267779102,
  119.19304750869061,
  247.05135754007196,
  312.6333333333333],
 7: [366.1687269572761,
  424.57063270990847,
  614.6663568773234,
  780.3933026920552,
  821.6944444444445],
 8: [704.31636205396,
  709.2108639005813,
  893.2548515314472,
  1009.6076948372246,
  1019.6123595505618],
 9: [781.4647293700089,
  787.3835699797161,
  879.0454008039726,
  985.4727212242183,
  990.7303370786517],
 10: [888.9103576405016,
  826.0633759902498,
  949.7732831608655,
  1007.

#### With this for loop I am displaying the avg pct change of the growth of the previous dict for each hour (the various if are used to limit the weight of outliers)

In [22]:
growth_light={}
for i in dicts.keys():
    if i in [1,2,3,4,5,19,20,21,22,22,23]:
        growth_light[i]=pd.Series(dicts[i])[0:-1].pct_change().mean()
    elif i in [7,8,18]:
        growth_light[i]=pd.Series(dicts[i])[-3:].pct_change().mean()
    elif i in [6]:
        growth_light[i]=pd.Series(dicts[i])[-2:].pct_change().mean()
    else:
        growth_light[i]=pd.Series(dicts[i]).pct_change().mean() 

In [23]:
multiplier = pd.DataFrame(growth_light.values())
multiplier.index = multiplier.index + 1
multiplier.reset_index(inplace=True)
multiplier.columns = ["hour","multiplier"]

In [24]:
grouped = light.drop(["week"],axis=1).groupby(["hour"],as_index=False).mean()

In [25]:
#new_df = pd.DataFrame(light.groupby(["zone","hour"]).mean()).reset_index()

# Movements

In [26]:
movement.reset_index(inplace=True)

In [27]:
movement["index"] = pd.to_datetime(movement["index"], errors="coerce")
movement["timestamp"] = pd.to_datetime(movement["timestamp"], errors="coerce")

In [28]:
movement["week"] = movement["index"].dt.isocalendar().week
movement["hour"] = movement["timestamp"].dt.hour

In [29]:
movement.drop(["index","timestamp","zone","sensor","id"],axis=1,inplace=True)

In [30]:
movements= movement.drop(["week"],axis=1).groupby(["hour"], as_index=False).sum()

#### Here we merge the passages per hour with the avg light per hour

In [31]:
import numpy as np
light_movement = pd.merge(grouped, movements, how="outer", on=["hour"])
light_movement["n_passages"].replace(np.nan,0, inplace=True)

In [32]:
light_movement["curtains_30%"] = grouped["value"]*0.7
light_movement["curtains_60%"] = grouped["value"]*0.4
light_movement["ALS"] = grouped["value"] + 450
light_movement.set_index(["hour"],inplace=True)

In [33]:
#### It's needed to iterate below to create the list "passages"
n_passages = light_movement["n_passages"].reset_index().copy()

### Forecasted Lux = (1 - Forecasted Cloud Level * 0,40) * Average Past Lux Distribution

In [34]:
forecast.rename(columns={"main/temp":"temperature","main/temp_min":"min_temperature",
                "main/temp_max":"max_temperature","main/pressure":"pressure",
               "main/sea_level":"sea_level", "main/grnd_level":"ground_level",
               "main/humidity":"humidity","weather/0/id":"weather_measure",
               "weather/0/main":"weather_cat","clouds/all":"cloud_measure",
               "dt_txt":"datetime","rain/3h":"rain_measure"}, inplace=True)

In [35]:
prevision.sort_values(["week","day"], inplace=True)

#### In the next passages I am trying to isolate the week from 18 to 22 of March (the most recent and complete) to the multiply it by the "multiplier". 

In [36]:
whole_week = prevision.groupby(["week","day","hour"],as_index=False).mean().copy()

In [37]:
whole_week = prevision[(prevision["day"]>=18)  & (prevision["day"]<=22)]

In [38]:
whole_week.drop(["week"],axis=1,inplace=True)

/Users/alessandro/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


#### Here I start creating "artificially" the forecasts for the values of light in April:
1) merge the whole week with the "multiplier"

2) adjusting both the date and the hour

3) multiplying the value of light by the multiplier

In [39]:
april = multiplier.merge(whole_week, on="hour", how="outer").sort_values(["day","hour"]).copy()

In [40]:
april["forecast"] = april["value"]*(1+april["multiplier"])*(1+april["multiplier"])

In [41]:
april["day"] = april["day"]-17
april["hour"] = april["hour"]+1

In [42]:
april.reset_index(inplace=True)
april.drop(["index","multiplier"],axis=1,inplace=True)

In [43]:
april = april.groupby(["day","hour"], as_index=False).mean()

In [44]:
forecast["datetime"] = pd.to_datetime(forecast["datetime"])

In [45]:
forecast["day"] = forecast["datetime"].dt.day
forecast["hour"] = forecast["datetime"].dt.hour

#### Here I merge the forecast.json df and the dataframe I created containing my forecast for april light measures.

In [46]:
final_df = forecast.merge(april, on=["day","hour"], how="inner")

In [47]:
final_df.drop(["dt","temperature","min_temperature",
               "max_temperature","pressure","sea_level",
               "ground_level","humidity","main/temp_kf",
               "weather_measure","weather_cat",
              "weather/0/description","weather/0/icon","wind/speed",
              "wind/deg","sys/pod","datetime","rain_measure","value"]
              ,axis=1, inplace=True)

In [48]:
#final_df = final_df.merge(passages, on=["hour"]).sort_values(["day","hour"])

In [49]:
final_df.reset_index(inplace=True)

In [50]:
final_df.drop(["index"], axis=1, inplace=True)

In [51]:
clouds=[]
for i in final_df.index:
    clouds.append(final_df.iloc[i,0])
    if i+1 in final_df.index:
        a = final_df.iloc[i+1,0]
        b = final_df.iloc[i,0]
        clouds.append("{:.2f}".format(b+(a-b)*0.33))
        clouds.append("{:.2f}".format(b+(a-b)*0.66))
    else:
        break

In [52]:
hours = []
for i in final_df.index:
    a = final_df.iloc[i,2]
    hours.append(a)
    if i+1 in final_df.index:
        hours.append(a+1)
        hours.append(a+2)  
    else:
        break

In [53]:
hours = pd.DataFrame(hours)
clouds = pd.DataFrame(clouds)
df_conc = pd.concat([clouds,hours],axis=1)
df_conc.columns = ["cloud_measure","hour"]

In [54]:
day = []
a=1
for i in df_conc.index:
    if i+1 in df_conc.index:
        if df_conc.index[i]==df_conc.index[0]:
            day.append(a)
        else:
            if df_conc.iloc[i,1] < df_conc.iloc[i+1,1]:
                day.append(a)   
            else:
                a+=1
                day.append(a)
    else:
        day.append(a)
        break

In [55]:
n_passage=[]
for i in df_conc.index:
    for h in n_passages.index:
        if df_conc.iloc[i,1] == n_passages.iloc[h,0]:
            n_passage.append(n_passages.iloc[h,1])

In [56]:
day = pd.DataFrame(day)
n_passages = pd.DataFrame(n_passage)

In [57]:
df_conc = pd.concat([day,hours,clouds,n_passages], axis=1)
df_conc.columns=["day","hour","cloud_measure","n_passages"]

In [58]:
df = df_conc.merge(april, on=["day","hour"],how="inner")
df.drop(["value"],axis=1,inplace=True)

## OPTIMIZATION

In [59]:
expenses = df.copy()

In [60]:
expenses["KWh_consumption_ALS_100%"]=0.5
expenses["pct_cloud_measure"] =pd.to_numeric(expenses.cloud_measure)
expenses["KWh_consumption_ALS_50%"]=0.3
expenses["€/KWh"] = 0.3
expenses["forecasted_lux"] = (1-(expenses.pct_cloud_measure/100)*0.4)*expenses.forecast
expenses["curtains_30%"] = expenses["forecasted_lux"]*0.7
expenses["curtains_60%"] = expenses["forecasted_lux"]*0.4
expenses["lux_with_100_ALS"] = expenses["forecasted_lux"] + 450
expenses["lux_with_50_ALS"] = expenses["forecasted_lux"] + 225

In [61]:
expenses.drop(["cloud_measure","diff","forecast","pct_cloud_measure"],axis=1,inplace=True)

In [62]:
best_option = []
for i in range(100):
    row = []
    for m in range(6,11):
        row.append(expenses.iloc[i,m])
    try:
        c = [x for x in row if (x<650) and (x>450)]
        best_option.append(min(c))
    except:
        newcol = []
        for k in range(len(row)):
            newcol.append(min([abs(450-row[k]),abs(row[k]-650)]))
        b = newcol.index(min(newcol))
        best_option.append(row[b])

In [63]:
best_option = pd.DataFrame(best_option)

In [64]:
expenses = pd.concat([expenses,best_option],axis=1)
expenses.rename(columns={0:"best_option"}, inplace=True)

In [65]:
names = []
for i in range(100):
    for m in range(6,11):
        if expenses.iloc[i,11] == expenses.iloc[i,m]:
            names.append(m)

In [66]:
names = pd.DataFrame(names)
expenses["Best_opt_name"] = expenses.columns[names]

<ipython-input-66-12756d2d813f>:2: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  expenses["Best_opt_name"] = expenses.columns[names]


In [67]:
for i in expenses.index:
    if expenses.loc[i,"n_passages"]==0.0:
        expenses.iloc[i,-1]="closed-lab"

In [68]:
expenses

,day,hour,n_passages,KWh_consumption_ALS_100%,KWh_consumption_ALS_50%,€/KWh,forecasted_lux,curtains_30%,curtains_60%,lux_with_100_ALS,lux_with_50_ALS,best_option,Best_opt_name
0,1,6,0.0,0.5,0.3,0.3,7.781399,5.446980,3.112560,457.781399,232.781399,457.781399,closed-lab
1,1,7,372.0,0.5,0.3,0.3,286.745976,200.722183,114.698390,736.745976,511.745976,511.745976,lux_with_50_ALS
2,1,8,1094.0,0.5,0.3,0.3,816.973911,571.881738,326.789564,1266.973911,1041.973911,571.881738,curtains_30%
3,1,9,2581.0,0.5,0.3,0.3,1086.277391,760.394174,434.510956,1536.277391,1311.277391,434.510956,curtains_60%
4,1,10,2476.0,0.5,0.3,0.3,918.510214,642.957149,367.404085,1368.510214,1143.510214,642.957149,curtains_30%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5,17,1128.0,0.5,0.3,0.3,828.327238,579.829066,331.330895,1278.327238,1053.327238,579.829066,curtains_30%
96,5,18,463.0,0.5,0.3,0.3,944.172956,660.921069,377.669183,1394.172956,1169.172956,660.921069,curtains_30%
97,5,19,144.0,0.5,0.3,0.3,209.907251,146.935076,83.962900,659.907251,434.907251,659.907251,lux_with_100_ALS
98,5,20,131.0,0.5,0.3,0.3,3.525243,2.467670,1.410097,453.525243,228.525243,453.525243,lux_with_100_ALS


In [69]:
cost = []
for i in range(100):
    #lack_of_confort
    if pd.Series(expenses.iloc[i,-2]).between(450,650)[0]==False:
        c = expenses.iloc[i,-2]
        cost.append(min([abs(c-450), abs(c-650)])*0.01)
    #cost_of_light
    else:
        if expenses.iloc[i,-1] in ["closed-lab",'curtains_30%','curtains_60%']:
            cost.append(0)
        elif expenses.iloc[i,-1] == "lux_with_50_ALS":
            cost.append(0.3*0.3)
        elif expenses.iloc[i,-1] == "lux_with_100_ALS":
            cost.append(0.5*0.3)
#cost = pd.DataFrame(cost)

## Automatic Curtains and New Neons

In [70]:
weekly_ammortization_neon=4000/365*7
weekly_ammortization_curtains=2000/365*7
total_cost_week = sum(cost)+weekly_ammortization_neon+weekly_ammortization_curtains
print(total_cost_week)

117.34780395509492


lack_of_comfort = abs(light_level-nearest_range)*0.01/h